# 🎤 OpenWakeWord Training - "Computer" Wake-Word

Dieses Notebook trainiert ein eigenes Wake-Word Modell mit OpenWakeWord.

**Voraussetzungen:**
- Aufnahmen auf Google Drive hochgeladen
- Ordnerstruktur: `wake_word_recordings/positive`, `negative`, `background_noise`

**Dauer:** 3-8 Stunden (je nach Anzahl der Samples)

## 📦 Schritt 1: Google Drive verbinden

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive verbunden!")

## 🔧 Schritt 2: OpenWakeWord installieren

In [ ]:
!pip install -q openwakeword tensorflow librosa soundfile
print("✅ OpenWakeWord installiert!")

## 📂 Schritt 3: Pfade zu deinen Aufnahmen angeben

**WICHTIG:** Passe die Pfade an, falls dein Ordner anders heißt!

In [ ]:
import os

# Pfade zu deinen Aufnahmen (ANPASSEN!)
BASE_PATH = "/content/drive/MyDrive/wake_word_recordings"
POSITIVE_PATH = os.path.join(BASE_PATH, "positive")
NEGATIVE_PATH = os.path.join(BASE_PATH, "negative")
BACKGROUND_PATH = os.path.join(BASE_PATH, "background_noise")

# Prüfe ob Ordner existieren
print(f"Positive Samples: {len(os.listdir(POSITIVE_PATH))} Dateien")
print(f"Negative Samples: {len(os.listdir(NEGATIVE_PATH))} Dateien")
print(f"Background Noise: {len(os.listdir(BACKGROUND_PATH))} Dateien")
print("\n✅ Alle Ordner gefunden!")

## 🎯 Schritt 4: Training vorbereiten

In [ ]:
import numpy as np
import librosa
import soundfile as sf
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

# Parameter
SAMPLE_RATE = 16000
DURATION = 2  # Sekunden
N_MELS = 40
N_FFT = 512
HOP_LENGTH = 160

print("✅ Training-Parameter gesetzt!")

## 🔊 Schritt 5: Audio-Dateien laden und vorverarbeiten

In [ ]:
def load_audio(file_path, duration=2, sr=16000):
    """Lädt Audio-Datei und normalisiert auf feste Länge."""
    try:
        audio, _ = librosa.load(file_path, sr=sr, duration=duration)
        # Pad oder trim auf exakte Länge
        target_length = sr * duration
        if len(audio) < target_length:
            audio = np.pad(audio, (0, target_length - len(audio)))
        else:
            audio = audio[:target_length]
        return audio
    except Exception as e:
        print(f"Fehler beim Laden von {file_path}: {e}")
        return None

def extract_melspectrogram(audio, sr=16000, n_mels=40, n_fft=512, hop_length=160):
    """Extrahiert Mel-Spektrogramm aus Audio."""
    mel_spec = librosa.feature.melspectrogram(
        y=audio, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length
    )
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db

print("✅ Audio-Funktionen definiert!")

## 📊 Schritt 6: Dataset erstellen

In [ ]:
def create_dataset(positive_path, negative_path, background_path):
    """Erstellt Dataset aus Aufnahmen."""
    X = []
    y = []
    
    # Positive Samples (Label 1)
    print("Lade positive Samples...")
    for i, filename in enumerate(os.listdir(positive_path)):
        if filename.endswith('.wav'):
            audio = load_audio(os.path.join(positive_path, filename))
            if audio is not None:
                mel_spec = extract_melspectrogram(audio)
                X.append(mel_spec)
                y.append(1)
        if (i + 1) % 100 == 0:
            print(f"  {i + 1} positive Samples geladen...")
    
    # Negative Samples (Label 0)
    print("Lade negative Samples...")
    for i, filename in enumerate(os.listdir(negative_path)):
        if filename.endswith('.wav'):
            audio = load_audio(os.path.join(negative_path, filename))
            if audio is not None:
                mel_spec = extract_melspectrogram(audio)
                X.append(mel_spec)
                y.append(0)
        if (i + 1) % 100 == 0:
            print(f"  {i + 1} negative Samples geladen...")
    
    # Background Noise (Label 0)
    print("Lade background noise...")
    for filename in os.listdir(background_path):
        if filename.endswith('.wav'):
            audio = load_audio(os.path.join(background_path, filename))
            if audio is not None:
                mel_spec = extract_melspectrogram(audio)
                X.append(mel_spec)
                y.append(0)
    
    X = np.array(X)
    y = np.array(y)
    
    print(f"\n✅ Dataset erstellt!")
    print(f"   Shape: {X.shape}")
    print(f"   Positive: {np.sum(y == 1)}")
    print(f"   Negative: {np.sum(y == 0)}")
    
    return X, y

# Dataset erstellen
X, y = create_dataset(POSITIVE_PATH, NEGATIVE_PATH, BACKGROUND_PATH)

## 🔀 Schritt 7: Train/Test Split

In [ ]:
# Reshape für CNN (Samples, Height, Width, Channels)
X = X[..., np.newaxis]

# Train/Test Split (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"✅ Dataset aufgeteilt!")
print(f"   Training: {X_train.shape[0]} Samples")
print(f"   Test: {X_test.shape[0]} Samples")

## 🧠 Schritt 8: Modell erstellen

In [ ]:
def create_model(input_shape):
    """Erstellt ein CNN-Modell für Wake-Word Detection."""
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(128, (3, 3), activation='relu'),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Modell erstellen
model = create_model(X_train.shape[1:])
model.summary()

print("\n✅ Modell erstellt!")

## 🚀 Schritt 9: Training starten!

**Das dauert jetzt 3-8 Stunden!** ⏳

Du kannst das Fenster schließen - das Training läuft weiter!

In [ ]:
# Callbacks
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7
)

# Training starten
print("🚀 Training startet...\n")

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\n✅ Training abgeschlossen!")

## 📊 Schritt 10: Ergebnisse anzeigen

In [ ]:
import matplotlib.pyplot as plt

# Accuracy Plot
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Test Accuracy
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n📊 Test Accuracy: {test_acc*100:.2f}%")
print(f"📊 Test Loss: {test_loss:.4f}")

## 💾 Schritt 11: Modell speichern

In [ ]:
# Modell auf Google Drive speichern
model_path = "/content/drive/MyDrive/computer_wake_word_model.h5"
model.save(model_path)

print(f"✅ Modell gespeichert: {model_path}")
print("\n📥 Du kannst es jetzt von Google Drive herunterladen!")

## 🎉 FERTIG!

**Dein "Computer" Wake-Word Modell ist fertig!**

**Nächste Schritte:**
1. Lade `computer_wake_word_model.h5` von Google Drive herunter
2. Kopiere es in dein Voice Assistant Projekt
3. Integriere es in den Code
4. Teste es!

**Viel Erfolg!** 🚀